# SyriaTel Customer Churn Prediction Model

## Imports

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, classification_report, plot_confusion_matrix
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.model_selection import KFold

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
from sklearn import set_config
set_config(display= 'diagram')

from imblearn.over_sampling import SMOTE, ADASYN

import warnings
warnings.filterwarnings('ignore')

## Original Data

In [ ]:
df_original = pd.read_csv('/Users/Arsh/Downloads/bigml_59c28831336c6604c800002a.csv')

In [ ]:
from copy import deepcopy
df = deepcopy(df_original)
# Keeping original dataset separate in case I need to come back to it.

## EDA

In [ ]:
df.head()
# I'm going to guess that there is no meaningful correlation between phone number and other features.

In [ ]:
df.shape

In [ ]:
df.dtypes
# Thankfully most columns are numerical and do not need to be changed. 

In [ ]:
df.isna().sum()
# There are no NaN values.

In [ ]:
df.state.value_counts()
# Does not seem to be important

In [ ]:
df['area code'].value_counts()
# Should be seen as a categorical column 

In [ ]:
df['international plan'].value_counts()
# Binary column - can be converted

In [ ]:
df['voice mail plan'].value_counts()
# Binary column - can be converted

In [ ]:
churn_num = df['churn'].value_counts()
churn_num
# Boolean column - does not need to be converted - also important- vry high class imbalance

In [ ]:
plt.title("Customer Churn Numbers")
plt.ylabel("Number")
churn_num.plot.bar()
# Class imbalance visualized

In [ ]:
plt.figure(figsize = (20,5))
sns.heatmap(df.corr().abs().loc[['churn'],:], annot = True)
# Not strong correlation between churn and other columns, closest thing is customer service calls, and daytime charges

In [ ]:
sns.pairplot(df, corner = True)
# Again no strong linear relationship between any variables that are visible from the pairplot

In [ ]:
sns.boxplot(x=df["churn"], y=df["customer service calls"]).set(title='Churn and Customer Service Calls')
# Clear relationship

In [ ]:
sns.boxplot( x=df["churn"], y=df["total day charge"]).set(title='Churn and Total Day Charge')
# Clear relationship - but less linear

In [ ]:
# Comparing the different costs for time of day, I suspect as day cost has higher correlation to churn, it is more expensive
day_cost = df['total day charge'].sum() / df['total day minutes'].sum()
eve_cost = df['total eve charge'].sum() / df['total eve minutes'].sum()
night_cost = df['total night charge'].sum() / df['total night minutes'].sum()
print(day_cost)
print(eve_cost)
print(night_cost)

In [ ]:
cost_info1 = {'Day Charge': 0.17000300739130672, 'Eve Charge': 0.0850010487148578, 'Night Charge': 0.04500041448440008}
cost_info2 = pd.DataFrame.from_dict(cost_info1, orient='index')
cost_info2

In [ ]:
cost_info2.plot.bar()
plt.suptitle("SyriaTel Charges by Time of Day")
plt.ylabel("Cost (in cents)")

## Baseline Model - Logistic Regression

In [ ]:
# This baseline will just use the numerical columns provided. Encoding the nominal columns, such as state and area code will be done later. 

In [ ]:
df_base = df.drop(columns = ['state', 'area code', 'phone number', 'international plan', 'voice mail plan'])

In [ ]:
y1 = df_base['churn']
X1 = df_base.drop('churn', axis=1)

# Split into training and test sets
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1, test_size=0.30, random_state=20)
# First test train split on this df, which has no categorical features

In [ ]:
steps = [('std_scaler', StandardScaler()),
        ('logreg', LogisticRegression(random_state=15))]

pipeline1 = Pipeline(steps)


# Train the pipeline (tranformations & predictor)
pipeline1.fit(X_train1, y_train1)

# Predict using the pipeline (includes the transfomers & trained predictor)
predicted1 = pipeline1.predict(X_test1)
predicted1

In [ ]:
print(classification_report(y_test1, predicted1))

In [ ]:
plot_confusion_matrix(pipeline1, X_test1, y_test1)

In [ ]:
# Ah. This makes sense, because of class imbalance, the model has trouble predicting True values

In [ ]:
# Time to do some data wrangling. 

## Data Wrangling

In [ ]:
# First I will convert all my object columns into numerical columns.

In [ ]:
df.loc[df['international plan'] == 'no', 'international_plan'] = 0
df.loc[df['international plan'] == 'yes', 'international_plan'] = 1
df['international_plan'] = df['international_plan'].astype(int)
df.loc[df['voice mail plan'] == 'no', 'voice_mail_plan'] = 0
df.loc[df['voice mail plan'] == 'yes', 'voice_mail_plan'] = 1
df['voice_mail_plan'] = df['voice_mail_plan'].astype(int)
df.loc[df['churn'] == False, 'churn_num'] = 0
df.loc[df['churn'] == True, 'churn_num'] = 1
df['churn_num'] = df['churn_num'].astype(int)

In [ ]:
df.head()

In [ ]:
# now the converted columns can be removed. 
df.drop(columns = ['phone number', 'international plan', 'voice mail plan', 'churn'], inplace = True)

In [ ]:
df.info()
# This is misleading, technically area code is a nominal category even though it has integer values.

In [ ]:
df['area code'] = df['area code'].astype(object) # Converting this column to an object so it can be run through a pipeline for nominal columns.

In [ ]:
df['area code'].dtypes #Just checking

## Model 2 - Random Forest

In [ ]:
# test train split on fixed df, also made random state number = number of rows
y = df['churn_num']
X = df.drop('churn_num', axis=1)

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=3333)

In [ ]:
# Am one-hot enoding all categorical columns, the others were binary so they were coverted to numerical ones
numeric_pipeline = Pipeline([('numnorm', StandardScaler())])

nominal_pipeline = Pipeline([
    ('onehotenc', OneHotEncoder(sparse = False, drop = 'first')), 
    ('onehotnorm', StandardScaler())])

In [ ]:
num_cols = X_train.select_dtypes(['int', 'float']).columns
ct = ColumnTransformer([("nominalpipe", nominal_pipeline, ['area code','state']),
     ("numpipe", numeric_pipeline, num_cols)])

num_cols

In [ ]:
X_train_scaled = pd.DataFrame(ct.fit_transform(X_train)).head()
X_train_scaled

In [ ]:
X_test_scaled = pd.DataFrame(ct.transform(X_test)).head()
X_test_scaled

In [ ]:
ct

In [ ]:
pipe2 = Pipeline([('preprocess', ct),
                      ('model',
                       RandomForestClassifier(random_state=25))])
pipe2

In [ ]:
pipe2.fit(X_train, y_train)

In [ ]:
len(X_test_scaled.columns) #Seeing the number of dummy columns created

In [ ]:
y_pred = pipe2.predict(X_test)

In [ ]:
plot_confusion_matrix(pipe2, X_test, y_test)

In [ ]:
print(classification_report(y_test, y_pred))

In [ ]:
# Much better than I expected, though the Random Forest model is having trouble prediciting True or 1 values in the target column.

### GridSearchCV
Hypertuning the Random Forest model

In [ ]:
rf_parameters = {'model__n_estimators': [20, 30, 40, 50, 100], 
             'model__min_samples_leaf': [1, 3, 5], 
             'model__max_depth': [4, 6, 8, 10, 12], 
             'model__min_samples_split': [1, 2, 5, 10, 12]} 
rf_cv = GridSearchCV(estimator = pipe2, param_grid = rf_parameters, cv = 5)
rf_cv.fit(X_train, y_train)

In [ ]:
rf_cv.best_score_

In [ ]:
rf_cv.best_params_

In [ ]:
tuned_model = rf_cv.best_estimator_
tuned_model

In [ ]:
tuned_model.fit(X_train, y_train)

In [ ]:
tuned_y_pred = tuned_model.predict(X_test)
print(classification_report(y_test,tuned_y_pred))
# How is this worse than the baseline?

In [ ]:
feat_imp = tuned_model['model'].feature_importances_

feat_imp_series = pd.Series(feat_imp, 
          index = X_test_scaled.columns).sort_values(
    ascending = False)

In [ ]:
feat_imp_series.head()
# This is unreadable

In [ ]:
figsize = (20,20)
feat_imp_series.plot(kind = 'bar')
# This is awful AND unreadable

### SMOTE
Perhaps using SMOTE will help with the class imbalance in the target column

In [ ]:
from imblearn.pipeline import Pipeline
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler

In [ ]:
tuned_model = RandomForestClassifier(max_depth=12, n_estimators=40, random_state=25)
over = SMOTE(sampling_strategy=0.25)
under = RandomUnderSampler(sampling_strategy=0.5)
steps = [('preprocess', ct), ('over', over), ('under', under), ('model', tuned_model)]
smote_pipeline = Pipeline(steps=steps)
smote_pipeline

In [ ]:
smote_pipeline.fit(X_train, y_train);
smote_y_pred = smote_pipeline.predict(X_test)
print(classification_report(y_test, smote_y_pred))

In [ ]:
plot_confusion_matrix(smote_pipeline, X_test, y_test)
# Much better, though I did sacrifice some precision

## Testing new idea
Testing an idea that Categorical Features have little predictive effect on customer churn. So I will rerun randomforest model again from the top.

In [ ]:
df2 = deepcopy(df_original)

In [ ]:
df2

In [ ]:
# Rerunning everthing again to maintain continuity
df2.drop(columns=['state', 'area code', 'phone number'], inplace = True)
df2.loc[df2['international plan'] == 'no', 'international_plan'] = 0
df2.loc[df2['international plan'] == 'yes', 'international_plan'] = 1
df2['international_plan'] = df2['international_plan'].astype(int)
df2.loc[df2['voice mail plan'] == 'no', 'voice_mail_plan'] = 0
df2.loc[df2['voice mail plan'] == 'yes', 'voice_mail_plan'] = 1
df2['voice_mail_plan'] = df2['voice_mail_plan'].astype(int)

In [ ]:
df2.drop(columns=['international plan', 'voice mail plan'], inplace = True)

In [ ]:
df2

In [ ]:
df2.dtypes
# Back to where we started

### New Random Forest Model - No Categoricals

In [ ]:
y2 = df2['churn']
X2 = df2.drop('churn', axis=1)

# Split into training and test sets
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, stratify = y2, test_size=0.30, random_state=15)

In [ ]:
steps = [('std_scaler', StandardScaler()),
        ('new_base', RandomForestClassifier(random_state=20))]

new_base = Pipeline(steps)


new_base.fit(X_train2, y_train2)

# Predict using the pipeline (includes the transfomers & trained predictor)
new_y_pred = new_base.predict(X_test2)

In [ ]:
print(classification_report(y_test2, new_y_pred))
# This is so much better ... and I did not even tune the model

In [ ]:
plot_confusion_matrix(new_base, X_test2, y_test2)
# Again so much better for a baseline

In [ ]:
new_base

### Hypertuning the new model

In [ ]:
new_parameters = {'new_base__n_estimators': [50, 100, 150, 200, 250], 
             'new_base__min_samples_leaf': [1, 3, 5, 9], 
             'new_base__max_features': [4, 5, 6, 8]} 
rf_cv2 = GridSearchCV(estimator = new_base, param_grid = new_parameters, cv = 5)
rf_cv2.fit(X_train2, y_train2)

In [ ]:
rf_cv2.best_score_

In [ ]:
rf_cv2.best_params_

In [ ]:
best_rf_model = rf_cv2.best_estimator_
best_rf_model

### Final Model - Hypertuned Random Forest

In [ ]:
best_rf_model.fit(X_train2, y_train2)
best_rf_model

In [ ]:
final_y_pred = best_rf_model.predict(X_test2)
print(classification_report(y_test2, final_y_pred ))
# Slightly better but still better

In [ ]:
plot_confusion_matrix(best_rf_model, X_test2, y_test2)

In [ ]:
feat_imp = best_rf_model['new_base'].feature_importances_

In [ ]:
feat_imp_series = pd.Series(feat_imp, 
          index = X_train2.columns).sort_values(
    ascending = False)
feat_imp_series

In [ ]:
feat_imp_series.plot(kind = 'barh')
# Validates the correlation heat map

### Extra Trees Classifier

In [ ]:
# Just trying this out...not great, but I still wanted to see if it worked
from sklearn.ensemble import ExtraTreesClassifier
etc = ExtraTreesClassifier(max_features='sqrt',
                         max_samples=0.5,
                         bootstrap=True,
                         random_state=1)

In [ ]:
etc.fit(X_train2, y_train2)

In [ ]:
etc.score(X_test2, y_test2)

In [ ]:
y_etc_pred = etc.predict(X_test2)
print(classification_report(y_test2, y_etc_pred))

In [ ]:
plot_confusion_matrix(etc, X_test2, y_test2)
# Not very good at predicting true positives